# Exploratory Data Analysis for Machine Learning

## Brief Description of the data
The bank manager is concerned that more and more customers are dropping out of credit card services. He would really appreciate it if someone could analyze the data to find out the reason for the churn. Thus, the bank could use this analysis to predict customers who are likely to be churned out so they could proactively reach out to these customers and provide them with better services, which would likely change their minds.

The dataset was taken from [Kaggle](https://www.kaggle.com/sakshigoyal7/credit-card-customers?select=BankChurners.csv) and contains 10127 different rows, each one representing a unique client.

The author recommends to drop the last two columns, since they do not provide any useful information for the study:
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_cou nt_Education_Level_Months_Inactive_12_mon_1
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_cou nt_Education_Level_Months_Inactive_12_mon_2

The remaining 21 columns are as follows:

| Column | Description |
| --- | --- |
| CLIENTNUM |  Client identification number |
| Attrition\_Flag |  Weather the customer account has been closed |
| Customer\_Age |  Age of a customer |
| Gender | Customer gender |
| Dependent\_Count | Number of dependents |
| Education\_Level | Education Qualification of account holder |
| Marital\_Status | Married, single, divorced, or unknown |
| Income\_Category | Annual income category of a customer |
| Card\_Category | Type of card |
| Month\_on\_book | Period of relationship with the bank |
| Total\_Relationship\_Count | Total number of products hold by the customer |
| Month\_Inactive\_12\_mon | Number of months inactive in the last 12 months |
| Contacts\_Count\_12\_mon | Number of contacts in the last 12 months |
| Credit\_Limit | Credit limit on the credit card |
| Total\_Revolving\_Bal | Total revolving balance on the card |
| Avg\_Open\_To\_Buy | Last 12 months average of open to buy credit line |
| Total\_Amt\_Chng\_Q4\_Q1 | Change in transaction amount (Q4 over Q1) |
| Total\_Trans\_Amt | Total transaction amount in last 12 months |
| Total\_Trans\_Ct | Total transaction count in last 12 months |
| Total\_Ct\_Chng\_Q4\_Q1 | Change in transaction count (Q4 over Q1) |
| Avg\_Utilization\_Ratio | Average card utilization ratio |

I also decided to drop the first column, since it doesn't provide any useful information either:
-  CLIENTNUM

## Data Summary

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']

# Import the libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wn
from scipy import stats

sns.set(style='darkgrid', palette='muted')

wn.filterwarnings('ignore')

In [ ]:
# Load the data
data = pd.read_csv('BankChurners.csv')

In [ ]:
# Remove Naive Bayes Classifier columns as a part of the dataset author recommendation
# Remove the Customer Number sinse it doesn't provide any useful information
data.drop(
    columns=[
             'CLIENTNUM',
             'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
             'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
    ],
    inplace=True
)

In [ ]:
# Quick look at the data
data.head()

In [ ]:
# Examine the data
data.info()

In [ ]:
# Confirm that there is no missing values
data.isnull().sum()

In [ ]:
# Confirm that there is no duplicated values
data.duplicated().sum()

## Data Cleaning and Exploratory Analysis

In [ ]:
# Create a copy of the original data
df = data.copy()

### Examine the summary statistics of the data

In [ ]:
# Examine the summary statistics of the numerical features
df.describe().T

### Distribution of Numerical Features

In [ ]:
# Get a smaller dataframe consisting of all the numericals
num_cols = df.select_dtypes(include=['int64', 'float64'])

In [ ]:
# Distribution of numerical features
num_cols.hist(bins=25, figsize=(14, 14))

### Outliers Handling

In [ ]:
# Visualize the numerical features to find outliers
plt.figure(figsize=(20, 20))
for i, col in enumerate(num_cols):
    plt.subplot(4, 4, i+1)
    sns.boxplot(data=df, x=col)

In [ ]:
def bounds(x):
    '''
    This funciton finds the lower and upper bounds for outliers in an array using the
    1.5IQR rule
    Input: x(array)
    Output: IQR, lower bound, upper bound
    '''
    q1 = np.quantile(x, 0.25)
    q3 = np.quantile(x, 0.75)
    iqr = q3 - q1

    # Finding upper and lower whiskers
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    return [iqr, lower, upper]

In [ ]:
for col in num_cols:
    # Using the 1.5 IQR rule to find outliers
    (iqr, lower, upper) = bounds(df[col])
    # Adjust values
    df[col] = df[col].apply(lambda x: upper if x > upper else (lower if x < lower else x))

In [ ]:
# Confirm that there are no more outliers in the numerical features
plt.figure(figsize=(20, 20))
for i, col in enumerate(num_cols):
    plt.subplot(4, 4, i+1)
    sns.boxplot(df[col])

### Distribution of Categorical Features

In [ ]:
# Examine the summary statistics of the non-numerical features
df.describe(include='O').T

In [ ]:
# Get a smaller dataframe consisting of all the string categoricals
cat_cols = df.select_dtypes(exclude=['int64', 'float64'])

In [ ]:
# Visualize the label
sns.countplot(data=cat_cols, x='Attrition_Flag')

In [ ]:
# Examine the value counts of the label
cat_cols['Attrition_Flag'].value_counts()

In [ ]:
# Visualize the gender
sns.countplot(data=cat_cols, x='Gender')

In [ ]:
# Examine the value counts of the gender
cat_cols['Gender'].value_counts()

In [ ]:
# Get a smaller dataframe consisting of all the other categorical features
cat_cols.drop(columns=['Attrition_Flag', 'Gender'], inplace=True)

In [ ]:
# Visualize the categorical features
plt.figure(figsize=(20, 12))
for i, col in enumerate(cat_cols):
    plt.subplot(2, 2, i+1)
    sns.countplot(data=cat_cols, x=col)

In [ ]:
# Examine the value counts of the categorical features
for col in cat_cols:
    print(f'{col}\n{cat_cols[col].value_counts()}\n')

## Feature Engineering

### Binary Encoding

In [ ]:
# Do binary encoding to the binary columns
df['Attrition_Flag'].replace({'Existing Customer': 0, 'Attrited Customer': 1}, inplace=True)
df['Gender'].replace({'M': 0, 'F': 1}, inplace=True)

### Ordinal Encoding

In [ ]:
# Do ordinal encoding to the ordinal columns
df['Marital_Status'].replace({'Married': 0, 'Single': 1, 'Divorced': 2, 'Unknown': 3}, inplace=True)
df['Card_Category'].replace({'Blue': 0, 'Silver': 1,  'Gold': 2, 'Platinum': 3}, inplace=True)

#### One Hot Encoding

In [ ]:
# Do the one hot encoding to categorical columns
ohe_cols = df.dtypes[df.dtypes == object]
ohe_cols = ohe_cols.index.tolist()
df = pd.get_dummies(df, columns=ohe_cols, drop_first=True)

### Log Transformation of the Skewed Features

In [ ]:
# Create a list of float colums to check for skewing
float_cols = df.select_dtypes(include=['float64'])

skew_limit = 0.75 # define a limit above which we will log transform
skew_vals = float_cols.skew()

In [ ]:
# Showing the skewed columns
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

skew_cols

In [ ]:
# Visualize skewed features before log1p transformation
plt.figure(figsize=(20, 5))
plt.suptitle('Before Log Transformation')
for i, col in enumerate(skew_cols.index):
    plt.subplot(1, 3, i+1)
    sns.histplot(data=float_cols, x=col)

In [ ]:
# Visualize skewed features after log1p transformation
plt.figure(figsize=(20, 5))
plt.suptitle('After Log Transformation')
for i, col in enumerate(skew_cols.index):
    plt.subplot(1, 3, i+1)
    sns.histplot(data=np.log1p(float_cols), x=col)

In [ ]:
# Perform the skew transformation on the data
for col in skew_cols.index.values:
    if col == 'Attrition_Flag':
        continue
    df[col] = df[col].apply(np.log1p)

### Pair Plots and Correlation Maps of the Features

In [ ]:
# Create a new copy of the processed dataset
X = df.copy()

In [ ]:
# Visualize pair plots of the features (full)
pair_cols = [
    'Attrition_Flag', 'Customer_Age', 'Dependent_count', 'Months_on_book',
    'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
    'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'
    ]
sns.pairplot(data=X[pair_cols], hue='Attrition_Flag', plot_kws={'alpha': 0.5})

In [ ]:
# Visualize correlation map of the features (full)
plt.figure(figsize=(15, 12))
sns.heatmap(X[pair_cols].corr(), annot=True)

In [ ]:

# Remove all correlated features from the data
X.drop(
        columns=[
                'Months_on_book', 'Avg_Open_To_Buy',
                'Total_Trans_Amt', 'Total_Trans_Ct',
                'Total_Revolving_Bal', 'Avg_Utilization_Ratio'
                ],
        inplace=True
        )

In [ ]:
# Visualize pair plots of the features (cleaned)
pair_cols = [
    'Attrition_Flag', 'Customer_Age', 'Dependent_count',
    'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
    'Credit_Limit', 'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1'
    ]
sns.pairplot(data=X[pair_cols], hue='Attrition_Flag', plot_kws={'alpha': 0.5})

In [ ]:
# Visualize correlation map of the features (cleaned)
plt.figure(figsize=(15, 12))
sns.heatmap(X[pair_cols].corr(), annot=True)

## Hypothesis Test

### Hypothesis 1:
There is a relationship between the number of dependents and a customer attrition

In [ ]:
# Linear Regression
x1 = X['Dependent_count']
y1 = X['Attrition_Flag']
x1 = sm.add_constant(x1)
model = sm.OLS(y1, x1)
results = model.fit()
results.summary()

### Hypothesis 2:
There is a relationship between the number of products hold by the customer and a customer attrition

In [ ]:
# Linear Regression
x2 = X['Total_Relationship_Count']
y2 = X['Attrition_Flag']
x2 = sm.add_constant(x2)
model = sm.OLS(y2, x2)
results = model.fit()
results.summary()

### Hypothesis 3:
Customer Age comes from a normal distribution

In [ ]:
# Normal Test
x3 = X['Customer_Age']
k2, p = stats.normaltest(x3)
k2, p

In [ ]:
# Save the processed data
X.to_csv('BankChurnersProcessed.csv', index=False)